In [ ]:
library(ppcor)
#install.packages('lmPerm')
#library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
#library(mediation)
#install.packages('caret')
library(caret)
library(dplyr)
#library(mediation)
#library(lavaan)
#install.packages('MissMech')
#library(semTools)
#library(MissMech)
#install.packages('fsbrain')
#install.packages('ipw')
#install.packages('episensr')
#install.packages('measurements')
library(tidyr)
#library(optimx)
#install.packages('R.matlab')
#library(devtools)
library(R.matlab)
library(lmPerm)
library('childsds')


# Set up PING data

In [ ]:
Brain_data <- readMat('/dagher/dagher11/filip/PING//CT_SA_CV_922_DKT_subjs.mat')

In [ ]:
brain_structs=c('caudalmiddlefrontal.l','entorhinal.l','postcentral.l','parstriangularis.l','supramarginal.l',
                'middlefilling.l','insula.l','lateralorbitofrontal.l','parsorbitalis.l','middletemporal.l',
                'pericalcarine.l','parahippocampal.l','paracentral.l','medialorbitofrontal.l','cuneus.l',
                'inferiortemporal.l','rostralmiddlefrontal.l','rostralanteriorcingulate.l','isthmuscingulate.l',
                'lateraloccipital.l','lingual.l','superiorparietal.l','parsopercularis.l','fusiform.l',
                'caudalanteriorcingulate.l','superiorfrontal.l','precuneus.l','transversetemporal.l','precentral.l',
                'inferiorparietal.l','posteriorcingulate.l','superiortemporal.l',
                'caudalmiddlefrontal.r','entorhinal.r','postcentral.r','parstriangularis.r','supramarginal.r',
                'middlefilling.r','insula.r','lateralorbitofrontal.r','parsorbitalis.r','middletemporal.r',
                'pericalcarine.r','parahippocampal.r','paracentral.r','medialorbitofrontal.r','cuneus.r',
                'inferiortemporal.r','rostralmiddlefrontal.r','rostralanteriorcingulate.r','isthmuscingulate.r',
                'lateraloccipital.r','lingual.r','superiorparietal.r','parsopercularis.r','fusiform.r',
                'caudalanteriorcingulate.r','superiorfrontal.r','precuneus.r','transversetemporal.r','precentral.r',
                'inferiorparietal.r','posteriorcingulate.r','superiortemporal.r')

brain_structs_vol=c('caudalmiddlefrontal.lv','entorhinal.lv','postcentral.lv','parstriangularis.lv','supramarginal.lv',
                'middlefilling.lv','insula.lv','lateralorbitofrontal.lv','parsorbitalis.lv','middletemporal.lv',
                'pericalcarine.lv','parahippocampal.lv','paracentral.lv','medialorbitofrontal.lv','cuneus.lv',
                'inferiortemporal.lv','rostralmiddlefrontal.lv','rostralanteriorcingulate.lv','isthmuscingulate.lv',
                'lateralccipital.lv','lingual.lv','superiorparietal.lv','parsopercularis.lv','fusiform.lv',
                'caudalanteriorcingulate.lv','superiorfrontal.lv','precuneus.lv','transversetemporal.lv','precentral.lv',
                'inferiorparietal.lv','posteriorcingulate.lv','superiortemporal.lv',
                'caudalmiddlefrontal.rv','entorhinal.rv','postcentral.rv','parstriangularis.rv','supramarginal.rv',
                'middlefilling.rv','insula.rv','lateralorbitofrontal.rv','parsorbitalis.rv','middletemporal.rv',
                'pericalcarine.rv','parahippocampal.rv','paracentral.rv','medialorbitofrontal.rv','cuneus.rv',
                'inferiortemporal.rv','rostralmiddlefrontal.rv','rostralanteriorcingulate.rv','isthmuscingulate.rv',
                'lateraloccipital.rv','lingual.rv','superiorparietal.rv','parsopercularis.rv','fusiform.rv',
                'caudalanteriorcingulate.rv','superiorfrontal.rv','precuneus.rv','transversetemporal.rv','precentral.rv',
                'inferiorparietal.rv','posteriorcingulate.rv','superiortemporal.rv')

brain_names=c(brain_structs,brain_structs_vol)

In [ ]:
CT=data.frame(Brain_data$ct)
CV=data.frame(Brain_data$cv)
colnames(CT)=brain_structs
colnames(CV)=brain_structs_vol

In [ ]:
Beh_data=read.table('/dagher/dagher11/filip/PING/PING_Behavior_All.csv',sep=',',skip=1)
Beh_data_cols=read.table('/dagher/dagher11/filip/PING/PING_Behavior_All.csv',sep=',')[1,]
colnames(Beh_data)=unlist(Beh_data_cols)
Beh_data$SubjID=as.numeric(gsub('P','',Beh_data$SubjID))

In [ ]:
FA=read.table('/dagher/dagher11/filip/PING/usercache_PING_nalsharif.csv',sep=',',quote='"',header=T)
FA$SubjID=as.numeric(gsub('P','',FA$SubjID))

In [ ]:
Brain=data.frame(Brain_data$subj.922,CT,CV)
All_data=merge(Brain,FA,by.x='Brain_data.subj.922',by.y='SubjID',all.x=T)
All_data=All_data[!duplicated(All_data$Brain_data.subj.922),]

In [ ]:
nrow(All_data)

In [ ]:
All_data=All_data[ All_data$FDH_2_White=='Yes', ]

# Set up ABCD data

In [ ]:
ABCD=read.table('/dagher/dagher11/filip/PRSOB/data/ABCD_base_fup1_long.csv',sep=',',dec='.', header=T)

## Remove duplicates

In [ ]:
ABCD=ABCD[!duplicated(ABCD$subjectkey),]

In [ ]:
ABCD=subset(ABCD, ABCD$demo_race_a_p___10==1)

## Remove all participants who failed WM QC for all sample consistency

In [ ]:
ABCD=subset(ABCD, ABCD$dmri_dti_postqc_qc==1 | is.na(ABCD$dmri_dti_postqc_qc))

In [ ]:
ABCD=ABCD[complete.cases(ABCD$H_dmri_dtifa_fiberat_fxrh),]

In [ ]:
PCs=read.table('/dagher/dagher11/filip/Downloads/ABCD_pre-impute.eigenvec', header=T)
ABCD=merge(ABCD, PCs, by.x='subjectkey', by.y='IID', all.x=T)
new_PRS=read.table('/dagher/dagher11/filip/Downloads/BMI_PRS_new.csv', header=T, sep=',')
colnames(new_PRS)=c( 'X', 'FID' ,'IID' ,'PRS', 'Z_UKB')
ABCD=merge(ABCD, new_PRS, by.x='subjectkey', by.y='IID', all.x=T)
ABCD$Z_corr=resid(lm(Z ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + site_id_l, 
                     data=ABCD, na.action=na.exclude))
ABCD$Z_corr_UKB=resid(lm(Z_UKB ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + site_id_l, 
                     data=ABCD, na.action=na.exclude))

## Read in raw data

In [ ]:
hdr <- read.table('/export01/data/fmorys/ABCD30/abcd_smrip101.txt', sep='\t',header=T, quote='"', stringsAsFactors = FALSE)
smri1 <- read.table('/export01/data/fmorys/ABCD30/abcd_smrip101.txt', sep='\t',header=F, quote='"', skip = 2)
colnames( smri1 ) <- colnames(hdr)

hdr <- read.table('/export01/data/fmorys/ABCD30/abcd_smrip201.txt', sep='\t',header=T, quote='"', stringsAsFactors = FALSE)
smri2 <- read.table('/export01/data/fmorys/ABCD30/abcd_smrip201.txt', sep='\t',header=F, quote='"', skip = 2)
colnames( smri2 ) <- colnames(hdr)

hdr <- read.table('/export01/data/fmorys/ABCD30/abcd_dti_p101.txt', sep='\t',header=T, quote='"', stringsAsFactors = FALSE)
dti <- read.table('/export01/data/fmorys/ABCD30/abcd_dti_p101.txt', sep='\t',header=F, quote='"', skip = 2)
colnames( dti ) <- colnames(hdr)
dti=subset(dti, select=-c(dataset_id, interview_date, collection_id, src_subject_id)) #Remove unnecessary columns for quicker merging further on


mri_all=merge(smri1, smri2)
mri_all=merge(mri_all, dti)

mri_all=mri_all[mri_all$eventname=='baseline_year_1_arm_1',]
#mri_all=subset(mri_all, select=c(subjectkey, smri_vol_scs_cbwmatterlh, smri_vol_scs_cbwmatterrh,
#                            smri_vol_cdk_total, smri_vol_scs_subcorticalgv, smri_vol_scs_allventricles)) #Remove unnecessary columns for quicker merging further on

In [ ]:
sbj=dplyr::select(ABCD, subjectkey)

In [ ]:
mri_all=merge(sbj, mri_all, by='subjectkey', all.x=T)

#  Predict based on global measures - GMV, WMV, sGMV, Ventricles

## Create dataframes only with necessary data

In [ ]:
PING_global=dplyr::select(All_data, Age, 
                          MRI_cort_vol.ctx.total, 
                          MRI_cort_thick.ctx.mean,
                          MRI_subcort_vol.IntracranialVolume)

for (i in 2:4) {
    
    PING_global[[i]]=resid(lm(PING_global[[i]] ~ factor(All_data$Gender, ordered=F) + 
                              factor(All_data$DeviceSerialNumber, ordered=F) + 
                              factor(All_data$FDH_3_Household_Income, ordered=F) + 
                              factor(All_data$FDH_Guardian_1_Edu, ordered=F) + 
                              factor(All_data$FDH_Guardian_2_Edu, ordered=F) +
                              factor(All_data$FDH_Highest_Education, ordered=F), 
                              na.action=na.exclude)) 
}

PING_global$MRI_cort_vol.ctx.total=PING_global$MRI_cort_vol.ctx.total/All_data$MRI_subcort_vol.IntracranialVolume

PING_global=na.omit(PING_global)

ABCD_global=dplyr::select(mri_all, smri_vol_cdk_total, 
                          smri_thick_cdk_mean,
                          smri_vol_scs_intracranialv)

for (i in 1:3) {
    
    ABCD_global[[i]]=resid(lm(ABCD_global[[i]] ~ factor(ABCD$sex, ordered=F) + 
                              factor(ABCD$site_id_l, ordered=F) +
                              factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                              factor(ABCD$demo_comb_income_v2, ordered=F) + 
                              factor(ABCD$demo_ed_v2, ordered=F) ,
                              factor(ABCD$demo_prnt_marital_v2, ordered=F), 
                              na.action=na.exclude)) 
}

ABCD_global$smri_vol_cdk_total=mri_all$smri_vol_cdk_total/mri_all$smri_vol_scs_intracranialv

ABCD_global=cbind(ABCD$age_yrs, ABCD_global, ABCD$Z_corr_UKB)
colnames(ABCD_global)=c(colnames(PING_global), 'BMIPRS')
ABCD_global=na.omit(ABCD_global)

In [ ]:
nrow(PING_global)

## Use linear and quadratic terms

In [ ]:
# Define training control
set.seed(123) 
train.control <- trainControl(method = "cv", number = 10, p=0.8)

#model_formula=as.formula(paste('Age ~',paste('poly(',colnames(PING_global[-1]),',2)',collapse = ' + ')))

# Train the model
#model <- train(model_formula, data = PING_global, method = "lm",
              # trControl = train.control)

model <- train(Age ~  MRI_cort_vol.ctx.total + 
                    MRI_cort_thick.ctx.mean , 
                    data = PING_global, method = "lm",
                    trControl = train.control)


# Summarize the results
print(model)

## Predict brain age in ABCD, calculate delta age and partially correlate with BMI-PRS correcting for chronological age

In [ ]:
ABCD_global$dAge=predict(model, ABCD_global)-ABCD_global$Age

summary(lmp(BMIPRS ~ dAge + 
            Age,
            data=ABCD_global, maxIter = 10000, center=F))

In [ ]:
tiff('/dagher/dagher11/filip/PRSOB/Figures/dAge_revision.tiff', width=20, height=10, units='cm', res=300)
ggplot(data=ABCD_global, aes(x=dAge, y=BMIPRS)) + 
geom_point(alpha=0.5) + 
geom_smooth(method='lm', color='black') + 
theme_minimal() +
xlab('Delta age') +
ylab('BMI-PRS')
dev.off()

## Brain age using specific measures

In [ ]:
CT_FS=dplyr::select(All_data,contains("MRI_cort_thick")[1:66])
CV_FS=dplyr::select(All_data,contains("MRI_cort_vol")[1:66])
FA_FS=dplyr::select(All_data,contains("DTI_fiber_FA.")[1:37])

In [ ]:
colnames(CT_FS)=c('l_bankssts','l_caudalanteriorcingulate','l_caudalmiddlefrontal','l_cuneus','l_entorhinal',
                  'l_fusiform','l_inferiorparietal','l_inferiortemporal','l_isthmuscingulate','l_lateraloccipital',
                  'l_lateralorbitofrontal','l_lingual','l_medialorbitofrontal','l_middletemporal','l_parahippocampal',
                  'l_paracentral','l_parsopercularis','l_parsorbitalis','l_parstriangularis','l_pericalcarine',
                  'l_postcentral','l_posteriorcingulate','l_precentral','l_precuneus','l_rostralanteriorcingulate',
                  'l_rostralmiddlefrontal','l_superiorfrontal','l_superiorparietal','l_superiortemporal','l_supramarginal',
                  'l_frontalpole','l_temporalpole','l_transversetemporal','r_bankssts','r_caudalanteriorcingulate',
                  'r_caudalmiddlefrontal','r_cuneus','r_entorhinal','r_fusiform','r_inferiorparietal','r_inferiortemporal',
                  'r_isthmuscingulate','r_lateraloccipital','r_lateralorbitofrontal','r_lingual','r_medialorbitofrontal',
                  'r_middletemporal','r_parahippocampal','r_paracentral','r_parsopercularis','r_parsorbitalis',
                  'r_parstriangularis','r_pericalcarine','r_postcentral','r_posteriorcingulate','r_precentral',
                  'r_precuneus','r_rostralanteriorcingulate','r_rostralmiddlefrontal','r_superiorfrontal',
                  'r_superiorparietal','r_superiortemporal','r_supramarginal','r_frontalpole','r_temporalpole',
                  'r_transversetemporal')
colnames(CT_FS)<-paste(colnames(CT_FS),"CT",sep="_")
CT_FS=scale(CT_FS)

colnames(CV_FS)=c('l_bankssts','l_caudalanteriorcingulate','l_caudalmiddlefrontal','l_cuneus','l_entorhinal',
                  'l_fusiform','l_inferiorparietal','l_inferiortemporal','l_isthmuscingulate','l_lateraloccipital',
                  'l_lateralorbitofrontal','l_lingual','l_medialorbitofrontal','l_middletemporal','l_parahippocampal',
                  'l_paracentral','l_parsopercularis','l_parsorbitalis','l_parstriangularis','l_pericalcarine',
                  'l_postcentral','l_posteriorcingulate','l_precentral','l_precuneus','l_rostralanteriorcingulate',
                  'l_rostralmiddlefrontal','l_superiorfrontal','l_superiorparietal','l_superiortemporal','l_supramarginal',
                  'l_frontalpole','l_temporalpole','l_transversetemporal','r_bankssts','r_caudalanteriorcingulate',
                  'r_caudalmiddlefrontal','r_cuneus','r_entorhinal','r_fusiform','r_inferiorparietal','r_inferiortemporal',
                  'r_isthmuscingulate','r_lateraloccipital','r_lateralorbitofrontal','r_lingual','r_medialorbitofrontal',
                  'r_middletemporal','r_parahippocampal','r_paracentral','r_parsopercularis','r_parsorbitalis',
                  'r_parstriangularis','r_pericalcarine','r_postcentral','r_posteriorcingulate','r_precentral',
                  'r_precuneus','r_rostralanteriorcingulate','r_rostralmiddlefrontal','r_superiorfrontal',
                  'r_superiorparietal','r_superiortemporal','r_supramarginal','r_frontalpole','r_temporalpole',
                  'r_transversetemporal')
colnames(CV_FS)<-paste(colnames(CT_FS),"CV",sep="_")
CV_FS=CV_FS/All_data$MRI_subcort_vol.IntracranialVolume
CV_FS=scale(CV_FS)

In [ ]:
CT_CV=cbind(CT_FS,CV_FS,FA_FS,All_data$Age_At_IMGExam)
colnames(CT_CV)=c(colnames(CT_CV[1:169]),'Age')
CT_CV=na.omit(CT_CV)

In [ ]:
CT_ABCD=dplyr::select(ABCD,contains("smri_thick_cdk")[c(1:33, 35:67)])
CV_ABCD=dplyr::select(ABCD,contains("smri_vol_cdk")[c(1:33,35:67)])
FA_ABCD=dplyr::select(ABCD, contains('dmri_dtifa_fiberat')[1:37])

In [ ]:
colnames(CT_ABCD)=c('l_bankssts','l_caudalanteriorcingulate','l_caudalmiddlefrontal','l_cuneus','l_entorhinal',
                  'l_fusiform','l_inferiorparietal','l_inferiortemporal','l_isthmuscingulate','l_lateraloccipital',
                  'l_lateralorbitofrontal','l_lingual','l_medialorbitofrontal','l_middletemporal','l_parahippocampal',
                  'l_paracentral','l_parsopercularis','l_parsorbitalis','l_parstriangularis','l_pericalcarine',
                  'l_postcentral','l_posteriorcingulate','l_precentral','l_precuneus','l_rostralanteriorcingulate',
                  'l_rostralmiddlefrontal','l_superiorfrontal','l_superiorparietal','l_superiortemporal','l_supramarginal',
                  'l_frontalpole','l_temporalpole','l_transversetemporal','r_bankssts','r_caudalanteriorcingulate',
                  'r_caudalmiddlefrontal','r_cuneus','r_entorhinal','r_fusiform','r_inferiorparietal','r_inferiortemporal',
                  'r_isthmuscingulate','r_lateraloccipital','r_lateralorbitofrontal','r_lingual','r_medialorbitofrontal',
                  'r_middletemporal','r_parahippocampal','r_paracentral','r_parsopercularis','r_parsorbitalis',
                  'r_parstriangularis','r_pericalcarine','r_postcentral','r_posteriorcingulate','r_precentral',
                  'r_precuneus','r_rostralanteriorcingulate','r_rostralmiddlefrontal','r_superiorfrontal',
                  'r_superiorparietal','r_superiortemporal','r_supramarginal','r_frontalpole','r_temporalpole',
                  'r_transversetemporal')
colnames(CT_ABCD)<-paste(colnames(CT_ABCD),"CT",sep="_")

colnames(CV_ABCD)=c('l_bankssts','l_caudalanteriorcingulate','l_caudalmiddlefrontal','l_cuneus','l_entorhinal',
                  'l_fusiform','l_inferiorparietal','l_inferiortemporal','l_isthmuscingulate','l_lateraloccipital',
                  'l_lateralorbitofrontal','l_lingual','l_medialorbitofrontal','l_middletemporal','l_parahippocampal',
                  'l_paracentral','l_parsopercularis','l_parsorbitalis','l_parstriangularis','l_pericalcarine',
                  'l_postcentral','l_posteriorcingulate','l_precentral','l_precuneus','l_rostralanteriorcingulate',
                  'l_rostralmiddlefrontal','l_superiorfrontal','l_superiorparietal','l_superiortemporal','l_supramarginal',
                  'l_frontalpole','l_temporalpole','l_transversetemporal','r_bankssts','r_caudalanteriorcingulate',
                  'r_caudalmiddlefrontal','r_cuneus','r_entorhinal','r_fusiform','r_inferiorparietal','r_inferiortemporal',
                  'r_isthmuscingulate','r_lateraloccipital','r_lateralorbitofrontal','r_lingual','r_medialorbitofrontal',
                  'r_middletemporal','r_parahippocampal','r_paracentral','r_parsopercularis','r_parsorbitalis',
                  'r_parstriangularis','r_pericalcarine','r_postcentral','r_posteriorcingulate','r_precentral',
                  'r_precuneus', 'r_rostralanteriorcingulate','r_rostralmiddlefrontal','r_superiorfrontal',
                  'r_superiorparietal','r_superiortemporal','r_supramarginal','r_frontalpole','r_temporalpole',
                  'r_transversetemporal')
colnames(CV_ABCD)<-paste(colnames(CT_ABCD),"CV",sep="_")

In [ ]:
CT_CV_ABCD=cbind(CT_ABCD, CV_ABCD, FA_ABCD, ABCD$age)
colnames(CT_CV_ABCD)=c(colnames(CT_CV_ABCD[1:132]),colnames(FA_FS),'Age')


In [ ]:
#CT_CV=CT_CV[,c(1:66,170)] #only CT
CT_CV=CT_CV[,c(1:132,170)] #only CV
#CT_CV=CT_CV[,c(6, 11, 15, 16, 19, 22, 23, 26, 30, 35, 42, 47, 49, 56, 60, 67)]
CT_CV=CT_CV[,c(35, 37, 32, 41, 11+66, 22+66, 23+66, 27+66, 35+66, 36+66, 38+66, 41+66, 44+66, 56+66, 59+66, 60+66, 67+66)]

#model_formula=as.formula(paste('Age ~',paste('poly(',colnames(CT_CV[1:14]),',2)',collapse = ' + ')))

# Train the model
#model <- train(model_formula, data = CT_CV, method = "lm",
#               trControl = train.control)



# Define training control
set.seed(123) 
train.control <- trainControl(method = "cv", number = 10)
# Train the model
model <- train(Age ~ ., data = CT_CV, method = "lm",
               trControl = train.control)
# Summarize the results
print(model)

In [ ]:
summary(model)

In [ ]:
CT_CV_ABCD$dAge=predict(model, CT_CV_ABCD)-CT_CV_ABCD$Age

In [ ]:
CT_CV_ABCD_Z=cbind(CT_CV_ABCD, ABCD$Z_corr_UKB)
colnames(CT_CV_ABCD_Z)=c(colnames(CT_CV_ABCD), 'BMIPRS')

In [ ]:
library(ppcor)

In [ ]:
pcor.test(CT_CV_ABCD_Z$BMIPRS, CT_CV_ABCD_Z$dAge, CT_CV_ABCD_Z$Age, method='pearson')